In [ ]:
!pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp39-cp39-linux_x86_64.whl size=3380635 sha256=f6e6c37a5c81983489cf7754340b23ba44540e600c32d6f400c5824bd7f4656d
  Stored in directory: /root/.cache/pip/wheels/db/81/9f/43a031fce121c845baca1c5d9a1468cad98208286aa2832de9
Successfully built mpi4py


In [ ]:
%%writefile vincenzo.py
#Carregando as Bibliotecas
import random
import sys
from mpi4py import MPI


random.seed(1)

# Gerador de combinações
def conv_base(num, base, exp):
    valor = num
    saida=[]
    while valor > 0:
        saida.append(valor % base)
        valor = valor // base
    while len(saida) < exp:
        saida.append(0)
    saida = [saida[len(saida)-i-1] for i in range(len(saida))]
    return saida

#Gerando o intervalo das combinações
def gera_inter(base, exp, inicio, fim):
    saida=[]
    for i in range(inicio, fim):
        saida.append(conv_base(i, base, exp))
    return(saida)

#Gerando a loja
def gera_loja(tam, max_preco, max_vol):
    loja=[]
    for i in range(tam):
        loja.append((random.randint(1, max_preco), random.randint(1, max_vol)))
    return loja

# Cálculo do custo e volume da combinação
def custo_vol(loja, comb):

    custo = 0
    vol = 0


    for i in range(len(loja)):
        custo += loja[i][0] * comb[i]
        vol += loja[i][1] * comb[i]

    return (custo, vol)



def div_lista(lista, qtd):
    tam = len(lista) // qtd # Tamanho de cada lista
    listas = []
    for t in range(qtd):
        inicio = t * tam # marca a posição inicial da lista
        fim = inicio + tam # marca a posição final da lista
        print(f'Intervalo {(t+1)}: {(inicio,fim-1)}')
        aux = []
        for i in range(inicio, fim):
            aux.append(lista[i])
        listas.append(aux)
        #listas.append([lista[i] for i in range(inicio, fim)])
        #print(t, qtd - 1, fim, len(lista)) #debug
        if t == qtd - 1:
            if fim < len(lista):
                for i in range(fim, len(lista)):
                    listas[t].append(lista[i])

    return listas



def main():

    #Iniciando o MPI
    comm = MPI.COMM_WORLD
    pid      = comm.Get_rank()          #PID do Processos atual
    numProcs = comm.Get_size()          #total de processos iniciados
    #MaqNome  = MPI.Get_processor_name() #Nome da máquina

    CAP_MOC = int(sys.argv[1])   # 30
    TAM_LOJA = int(sys.argv[2])  # 5
    QTD_PROD = int(sys.argv[3])  # 3
    MAX_PRECO = int(sys.argv[4]) #20
    MAX_VOL = int(sys.argv[5])   #10




    if pid == 0:

        loja = gera_loja(TAM_LOJA, MAX_PRECO, MAX_VOL)  #gera loja
        comb = gera_inter(QTD_PROD, TAM_LOJA, 0, QTD_PROD**TAM_LOJA)  #gera combinações
        print(QTD_PROD**TAM_LOJA)               #printa tamanho da loja
        listas = div_lista(comb, numProcs)      #divide a lista em varias listas de acordo com o numero de processos e execução inicializa uma lista vazia 'aux'

        print(len(listas))   #printa lista
        aux = []   #auxiliar para receber os melhores valores
#-----------------------------------------------------------------------------
        lista = listas[0]    #processo 0 recebe a primeira lista

        #print(f'pid ({pid})')
        #print(f'pid ({pid}) recebeu lista  ({lista})')

        #print(f'pid ({pid}) recebeu lista  ({lista})')

        max = (0, 0)
        max_comb = []   #salva a melhor combinação
        for teste in lista:     #calcula o melhor valor na primeira lista (do processo 0)
          #print(teste)
          resp = custo_vol(loja, teste)
          if resp[0] >= max[0] and resp[1] <= CAP_MOC:
              max = resp
              max_comb = teste

        #print(f'pid ({pid}) enviando {max_comb}')

        aux.append(max_comb)  #adiciona a melhor combinação do processo 0 à variavel auxiliar
        #print(aux)




        for proc in range(1, numProcs):   #envia a loja para todos processos exceto o 0
            comm.send(loja, dest=proc)

        for proc in range(1, numProcs):   #envia a lista de cada processo
            comm.send(listas[proc], dest=proc)




        for proc in range(1, numProcs):   #recebe as melhores combinações de cada processo
            aux.append(comm.recv(source=proc))

        print(aux)      #printa as melhores combinações

        max = (0, 0)
        max_comb = []

        for teste in aux:     #calcula a melhor combinação das melhores combinações recebidas dos processos

          #print(teste)
          resp = custo_vol(loja, teste)

          if resp[0] >= max[0] and resp[1] <= CAP_MOC:
              max = resp
              max_comb = teste


        print(loja)     #printa a loja

        print(max_comb)   #printa a melhor combinação
        print(max)      #printa os valores da melhor combinação

    else:

        loja = comm.recv(source=0)  #recebe a loja do processo 0
        lista = comm.recv(source=0)  #recebe a lista do processo 0


        #print(f'pid ({pid})')
        #print(f'pid ({pid}) recebeu lista  ({lista})')

        #print(f'pid ({pid}) recebeu lista  ({lista})')

        max = (0, 0)
        max_comb = []
        for teste in lista:   #realiza o teste da melhor combinação do processo atual
          #print(teste)
          resp = custo_vol(loja, teste)
          if resp[0] >= max[0] and resp[1] <= CAP_MOC:
              max = resp
              max_comb = teste

        #print(f'pid ({pid}) enviando {max_comb}')
        comm.send(max_comb, dest=0)


if __name__ == '__main__':
    main()

Overwriting vincenzo.py


In [ ]:
!mpirun --oversubscribe --allow-run-as-root -np 4 python vincenzo.py 30 5 4 20 10

1024
Intervalo 1: (0, 255)
Intervalo 2: (256, 511)
Intervalo 3: (512, 767)
Intervalo 4: (768, 1023)
4
[[0, 0, 0, 2, 3], [1, 0, 0, 1, 3], [2, 0, 0, 0, 2], [3, 0, 0, 0, 0]]
[(5, 10), (3, 5), (4, 8), (15, 8), (13, 4)]
[0, 0, 0, 2, 3]
(69, 28)
